In [ ]:
import pandas as pd
import keras as keras

In [ ]:
dates = pd.read_csv("./training_data/dates.csv")
test  = pd.read_csv("./training_data/zecon/City_time_series.csv")

In [ ]:
import recognize as rec

In [ ]:
dates_table = rec.Recognize(dates).make_table()

In [ ]:
test_date = pd.DataFrame(test.Date)
test_date =  rec.Recognize(test_date).make_table()

test_str = pd.DataFrame(test.MedianPctOfPriceReduction_CondoCoop)
test_str =  rec.Recognize(test_str).make_table()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dates_table.label)
dates_table.label = le.transform(dates_table.label)

le.fit(dates_table['mean_val'])
dates_table['mean_val'] = le.transform(dates_table['mean_val'])

In [ ]:
dates_table

In [ ]:
# Import `train_test_split` from `sklearn.model_selection`
from sklearn.model_selection import train_test_split

# Specify the data 
X= dates_table.loc[:,:'mean']

# Specify the target labels and flatten the array 
y=np.ravel(dates_table.loc[:,'label'])

# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [ ]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(95,)))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Model output shape
model.output_shape

# Model summary
model.summary()

# Model config
model.get_config()

# List all weight tensors 
model.get_weights()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=4, batch_size=1, verbose=1)

In [ ]:
y_pred = model.predict(X_test)
score = model.evaluate(X_test, y_test,verbose=1)

score

In [ ]:
test_str['mean'] = 0
td = pd.DataFrame(test_str.loc[3,:'mean'].values.reshape(-1,1)).transpose()
td.columns = columns

In [ ]:
model.predict(td)

In [ ]:
# Import the modules from `sklearn.metrics`
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

# Confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test,y_pred)

In [1]:
import sys, json
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

class Engine:
	'''
		file_name: client file to be processed
		json_out : ouput json object to be send to client
	'''
	def __init__(self, file_name):
		self.file_name = file_name
		self.json_out = {}
		self.dataset = pd.read_csv((file_name))

	def bin_this_df(self, df):#qcut each feature. 
		temp_df = df.copy()
		for i in temp_df.columns:
			try:
				temp_df.loc[:,i] = pd.qcut(temp_df.loc[:,i], q = 4)
			except Exception:
				pass
		return temp_df

	def apriori(self, dataset):
		temp_df = self.bin_this_df(dataset)
		data_list = temp_df.to_numpy()     
		te = TransactionEncoder()
		te_ary = te.fit(data_list).transform(data_list)
		te_ary_df = pd.DataFrame(te_ary, columns=te.columns_)
		min_support = 0.1
		frequent_itemsets = apriori(te_ary_df, min_support= min_support, use_colnames=True)
		while (len((frequent_itemsets)) < 3) & (min_support > 0.04):
			if min_support == 0.1:
				min_support -= 0.01
			else:
				min_support -= 0.1
			frequent_itemsets = apriori(te_ary_df, min_support= min_support,  use_colnames=True)
		return frequent_itemsets


	def assoc_rules(self, frequent_itemsets):
		rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
		frequent_itemsets.itemsets = frequent_itemsets.itemsets.apply(list)
		fi_itemsets = list(frequent_itemsets.itemsets)
		fi_support = list(frequent_itemsets.support)
		freq_dict = {}

		for i in range(0,len(fi_itemsets)-1):
			freq_dict[str(fi_support[i])] = fi_itemsets[i]
		if (len(frequent_itemsets) < 2):
			print("no frequent_itemsets")
		else:
			return freq_dict

#print((dataset.describe()).to_json())
    

ModuleNotFoundError: No module named 'mlxtend'

In [3]:
from mlxtend.frequent_patterns import apriori

ModuleNotFoundError: No module named 'mlxtend'